In [1]:
from __future__ import division, absolute_import
from __future__ import print_function, unicode_literals

import time
import numpy as np
import sklearn.datasets
import sklearn.cross_validation
import sklearn.metrics
import theano
import theano.tensor as T
import lasagne

# ############################### prepare data ###############################

mnist = sklearn.datasets.fetch_mldata('MNIST original')

X = mnist['data'].astype(np.float32) / 255.0
y = mnist['target'].astype("int32")
X_train, X_valid, y_train, y_valid = sklearn.cross_validation.train_test_split(
    X, y, random_state=42, train_size=50000, test_size=10000)
X_train = X_train.reshape(-1, 1, 28, 28)
X_valid = X_valid.reshape(-1, 1, 28, 28)

l_in = lasagne.layers.InputLayer(
    shape=(None, 1, 28, 28),
)

l_hidden1 = lasagne.layers.DenseLayer(
    l_in,
    num_units=64,
    nonlinearity=lasagne.nonlinearities.rectify,
    W=lasagne.init.GlorotUniform(),
)

l_out = lasagne.layers.DenseLayer(
    l_hidden1,
    num_units=10,
    nonlinearity=lasagne.nonlinearities.softmax,
    W=lasagne.init.GlorotUniform(),
)

# ############################### network loss ###############################

# int32 vector
target_vector = T.ivector('y')


def loss_fn(output):
    return T.mean(lasagne.objectives.categorical_crossentropy(output,
                                                              target_vector))

output = lasagne.layers.get_output(l_out)
loss = loss_fn(output)

# ######################## compiling theano functions ########################

print("Compiling theano functions")

# - takes out all weight tensors from the network, in order to compute
#   how the weights should be updated
all_params = lasagne.layers.get_all_params(l_out)

# - calculate how the parameters should be updated
# - theano keeps a graph of operations, so that gradients w.r.t.
#   the loss can be calculated
updates = lasagne.updates.sgd(
    loss_or_grads=loss,
    params=all_params,
    learning_rate=0.001)

# - create a function that also updates the weights
# - this function takes in 2 arguments: the input batch of images and a
#   target vector (the y's) and returns a list with a single scalar
#   element (the loss)
train_fn = theano.function(inputs=[l_in.input_var, target_vector],
                           outputs=[loss],
                           updates=updates)

# - same interface as previous the previous function, but now the
#   output is a list where the first element is the loss, and the
#   second element is the actual predicted probabilities for the
#   input data
valid_fn = theano.function(inputs=[l_in.input_var, target_vector],
                           outputs=[loss, output])

# ################################# training #################################

print("Starting training...")

num_epochs = 25
batch_size = 600
for epoch_num in range(num_epochs):
    num_batches_train = int(np.ceil(len(X_train) / batch_size))

    train_losses = []
    for batch_num in range(num_batches_train):
        X_batch = X_train[batch_size * batch_num:batch_size * (batch_num + 1)]
        y_batch = y_train[batch_size * batch_num:batch_size * (batch_num + 1)]

        loss, = train_fn(X_batch, y_batch)
        train_losses.append(loss)
    # aggregate training losses for each minibatch into scalar
    train_loss = np.mean(train_losses)

    # calculate validation loss
    num_batches_valid = int(np.ceil(len(X_valid) / batch_size))
    valid_losses = []
    list_of_probabilities_batch = []
    for batch_num in range(num_batches_valid):
        X_batch = X_valid[batch_size * batch_num:batch_size * (batch_num + 1)]
        y_batch = y_valid[batch_size * batch_num:batch_size * (batch_num + 1)]

        loss, probabilities_batch = valid_fn(X_batch, y_batch)
        valid_losses.append(loss)
        list_of_probabilities_batch.append(probabilities_batch)
    valid_loss = np.mean(valid_losses)
    # concatenate probabilities for each batch into a matrix
    probabilities = np.concatenate(list_of_probabilities_batch)
    # calculate classes from the probabilities
    predicted_classes = np.argmax(probabilities, axis=1)
    # calculate accuracy for this epoch
    accuracy = sklearn.metrics.accuracy_score(y_valid, predicted_classes)

    print("Epoch: %d, train_loss=%f, valid_loss=%f, valid_accuracy=%f"
          % (epoch_num + 1, train_loss, valid_loss, accuracy))

Compiling theano functions
Starting training...
Epoch: 1, train_loss=2.289660, valid_loss=2.239347, valid_accuracy=0.174100, time=0.152448s
Epoch: 2, train_loss=2.195213, valid_loss=2.157211, valid_accuracy=0.240400, time=0.143531s
Epoch: 3, train_loss=2.120240, valid_loss=2.088282, valid_accuracy=0.295600, time=0.141754s
Epoch: 4, train_loss=2.054930, valid_loss=2.026074, valid_accuracy=0.357000, time=0.146649s
Epoch: 5, train_loss=1.994707, valid_loss=1.967577, valid_accuracy=0.418100, time=0.170406s
Epoch: 6, train_loss=1.937376, valid_loss=1.911292, valid_accuracy=0.473400, time=0.148437s
Epoch: 7, train_loss=1.881857, valid_loss=1.856489, valid_accuracy=0.520700, time=0.144898s
Epoch: 8, train_loss=1.827674, valid_loss=1.802861, valid_accuracy=0.560600, time=0.142810s
Epoch: 9, train_loss=1.774583, valid_loss=1.750262, valid_accuracy=0.592000, time=0.143788s
Epoch: 10, train_loss=1.722589, valid_loss=1.698759, valid_accuracy=0.618800, time=0.146223s
Epoch: 11, train_loss=1.671765,

In [ ]:
# Visualizing things

In [2]:
def show_images(img_original, saliency):
    saliency = saliency[0]
    # convert saliency from BGR to RGB, and from c01 to 01c
    saliency = saliency[::-1].transpose(1, 2, 0)
    # plot the original image and the three saliency map variants
    plt.figure(figsize=(10, 10), facecolor='w')
    plt.subplot(2, 2, 1)
    plt.title('input')
    plt.imshow(img_original)
    plt.subplot(2, 2, 2)
    plt.title('abs. saliency')
    plt.imshow(np.abs(saliency).max(axis=-1), cmap='gray')
    plt.subplot(2, 2, 3)
    plt.title('pos. saliency')
    plt.imshow((np.maximum(0, saliency) / saliency.max()))
    plt.subplot(2, 2, 4)
    plt.title('neg. saliency')
    plt.imshow((np.maximum(0, -saliency) / -saliency.min()))
    plt.show()

In [3]:
class ModifiedBackprop(object):

    def __init__(self, nonlinearity):
        self.nonlinearity = nonlinearity
        self.ops = {}  # memoizes an OpFromGraph instance per tensor type

    def __call__(self, x):
        # OpFromGraph is oblique to Theano optimizations, so we need to move
        # things to GPU ourselves if needed.
        if theano.sandbox.cuda.cuda_enabled:
            maybe_to_gpu = theano.sandbox.cuda.as_cuda_ndarray_variable
        else:
            maybe_to_gpu = lambda x: x
        # We move the input to GPU if needed.
        x = maybe_to_gpu(x)
        # We note the tensor type of the input variable to the nonlinearity
        # (mainly dimensionality and dtype); we need to create a fitting Op.
        tensor_type = x.type
        # If we did not create a suitable Op yet, this is the time to do so.
        if tensor_type not in self.ops:
            # For the graph, we create an input variable of the correct type:
            inp = tensor_type()
            # We pass it through the nonlinearity (and move to GPU if needed).
            outp = maybe_to_gpu(self.nonlinearity(inp))
            # Then we fix the forward expression...
            op = theano.OpFromGraph([inp], [outp])
            # ...and replace the gradient with our own (defined in a subclass).
            op.grad = self.grad
            # Finally, we memoize the new Op
            self.ops[tensor_type] = op
        # And apply the memoized Op to the input we got.
        return self.ops[tensor_type](x)


In [4]:
class GuidedBackprop(ModifiedBackprop):
    def grad(self, inputs, out_grads):
        (inp,) = inputs
        (grd,) = out_grads
        dtype = inp.dtype
        return (grd * (inp > 0).astype(dtype) * (grd > 0).astype(dtype),)

In [ ]:
relu = lasagne.nonlinearities.rectify
relu_layers = [layer for layer in lasagne.layers.get_all_layers(net['prob'])
               if getattr(layer, 'nonlinearity', None) is relu]
modded_relu = GuidedBackprop(relu)  # important: only instantiate this once!
for layer in relu_layers:
    layer.nonlinearity = modded_relu